<a href="https://colab.research.google.com/github/rahadiyant/lulusan/blob/main/lulusan01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install streamlit

In [ ]:
!pip install pandas


In [ ]:
!pip install matplotlib

In [ ]:
!pip install seaborn

In [ ]:
!pip install pyngrok

In [ ]:
import yaml
from pyngrok import ngrok, conf

# Load the ngrok configuration
ngrok_config = '''
version: '2'
authtoken: 2iJyVmdEVq1QrJmRETFGsDMgL9W_3W221G4nNav1CVd6ggxBJ
region: us
'''  # Replace YOUR_NGROK_AUTHTOKEN with your actual authtoken

# Parse the configuration
config = yaml.safe_load(ngrok_config)

# Set the configuration
conf.get_default().config_path = config

# Connect to ngrok with the correct port configuration
public_url = ngrok.connect(8501)
print('Ngrok tunnel opened at:', public_url)

In [ ]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.186s


In [ ]:
!pip install plotly


In [ ]:
!pip install scikit-learn

In [40]:
%%writefile lulusan.py
import streamlit as st
from datetime import time, date
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
import numpy as np
from pyngrok import ngrok
import subprocess

# Set style untuk seaborn
sns.set_style("darkgrid")

# Definisikan aplikasi Streamlit
def run():
    st.title("Data Lulusan STIKES HAKLI Semarang")

    st.markdown("""
        Dataset ini berisi informasi tentang Jumlah lulusan mahasiswa STIKES HAKLI Semarang.

        ## Pernyataan Masalah
        Kita ingin menganalisis Jumlah Lulusan mahasiswa dan membangun model prediktif untuk beberapa parameter mahasiswa.
    ### Deskripsi Data
        - No
        - Tahun
        - Kesling
        - RMIK
        - KESMAS
        - RO
    """)

    st.sidebar.title("Operasi pada Dataset")

    w1 = st.sidebar.checkbox("Tampilkan tabel", False)
    plot = st.sidebar.checkbox("Tampilkan plot", False)
    plothist = st.sidebar.checkbox("Tampilkan plot histogram", False)
    trainmodel = st.sidebar.checkbox("Latih model", False)
    dokfold = st.sidebar.checkbox("Lakukan KFold", False)
    distView = st.sidebar.checkbox("Tampilan Distribusi", False)
    _3dplot = st.sidebar.checkbox("Plot 3D", False)
    linechart = st.sidebar.checkbox("Grafik Garis", False)
    show_graduates = st.sidebar.checkbox("Tampilkan jumlah lulusan", False)

    @st.cache_data
    def read_data():
        return pd.read_excel("https://github.com/rahadiyant/lulusan/raw/main/Data_Lulusan_stikes.xlsx")

    df = read_data()

    if w1:
        st.dataframe(df)

    if show_graduates:
        st.subheader("Jumlah Lulusan Per Tahun dan Total Lulusan")
        df['Total'] = df[['Kesling', 'RMIK', 'KESMAS', 'RO']].sum(axis=1)
        graduates_per_year = df.groupby('Tahun')['Total'].sum()
        total_graduates = df['Total'].sum()

        st.write("Jumlah Lulusan Per Tahun:")
        st.write(graduates_per_year)

        st.write(f"Total Lulusan: {total_graduates}")

    if linechart:
        st.subheader("Grafik Garis")
        numeric_df = df.select_dtypes(include=[np.number])  # Only numerical columns
        st.line_chart(numeric_df)

    if plothist:
        st.subheader("Distribusi Setiap Kolom")
        options = df.columns.tolist()
        sel_cols = st.selectbox("Pilih kolom", options, 0, key="plothist_selectbox")
        fig = go.Figure(data=[go.Histogram(x=df[sel_cols], nbinsx=50)])
        st.plotly_chart(fig)

    if plot:
        st.subheader("Korelasi antara Parameter")
        options = df.select_dtypes(include=[np.number]).columns.tolist()
        w7 = st.selectbox("Pilih parameter", options, 0, key="plot_selectbox")

        fig, ax = plt.subplots()
        ax.scatter(df.index, df[w7])
        ax.set_xlabel("Index")
        ax.set_ylabel(w7)
        ax.set_title(f"Scatter Plot: {w7}")
        st.pyplot(fig)

    if distView:
        st.subheader("Tampilan Distribusi Gabungan")
        numeric_df = df.select_dtypes(include=[np.number])  # Only numerical columns
        columns_to_plot = st.multiselect('Pilih kolom untuk ditampilkan dalam distribusi', numeric_df.columns.tolist(), key="distView_multiselect")
        if columns_to_plot:
            hist_data = [numeric_df[col].dropna().values for col in columns_to_plot]
            group_labels = columns_to_plot
            fig = ff.create_distplot(hist_data, group_labels, bin_size=[.1]*len(hist_data))
            st.plotly_chart(fig)

    if _3dplot:
        numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(numeric_columns) >= 3:
            options = st.multiselect('Pilih kolom untuk plot', numeric_columns, numeric_columns[:3], key="3dplot_multiselect")
            st.write('Anda memilih:', options)
            if len(options) >= 3:
                hist_data = [df[col].dropna().values for col in options]
                trace1 = go.Scatter3d(
                    x=hist_data[0],
                    y=hist_data[1],
                    z=hist_data[2] if len(hist_data) > 2 else hist_data[0],
                    mode="markers",
                    marker=dict(size=8, colorscale="Viridis"),
                )
                data = [trace1]
                layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
                fig = go.Figure(data=data, layout=layout)
                st.plotly_chart(fig)
            else:
                st.error("Pilih setidaknya tiga kolom untuk plot 3D.")
        else:
            st.error("Tidak cukup kolom numerik untuk plot 3D.")

    if trainmodel:
        st.header("Pemodelan")
        target_column = st.selectbox("Pilih kolom target", df.columns, key="trainmodel_selectbox")
        if target_column in df.columns:
            y = df[target_column]
            X = df.drop(columns=[target_column]).select_dtypes(include=[np.number]).values
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            lrgr = LinearRegression()
            lrgr.fit(X_train, y_train)
            pred = lrgr.predict(X_test)
            mse = mean_squared_error(y_test, pred)
            rmse = sqrt(mse)
            st.markdown(f"""
            Model Regresi Linear telah dilatih :
                - MSE: {mse}
                - RMSE: {rmse}
            """)
            st.success('Model berhasil dilatih')
        else:
            st.error(f"Kolom '{target_column}' tidak ditemukan dalam dataset.")

    if dokfold:
        st.subheader("Evaluasi KFold Random Sampling")
        target_column = st.selectbox("Pilih kolom target", df.columns, key="dokfold_selectbox")
        if target_column in df.columns:
            my_bar = st.progress(0)
            X = df.drop(columns=[target_column]).select_dtypes(include=[np.number]).values
            y = df[target_column].values
            kf = KFold(n_splits=10, shuffle=True, random_state=42)
            mse_list = []
            rmse_list = []
            r2_list = []
            idx = 1
            fig, ax = plt.subplots()
            for train_index, test_index in kf.split(X):
                my_bar.progress(idx * 10)
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                lrgr = LinearRegression()
                lrgr.fit(X_train, y_train)
                pred = lrgr.predict(X_test)
                mse = mean_squared_error(y_test, pred)
                rmse = sqrt(mse)
                r2 = r2_score(y_test, pred)
                mse_list.append(mse)
                rmse_list.append(rmse)
                r2_list.append(r2)
                ax.plot(pred, label=f"dataset-{idx}")
                idx += 1
            ax.legend()
            ax.set_xlabel("Indeks Data")
            ax.set_ylabel("Nilai Prediksi")

            st.pyplot(fig)
            res = pd.DataFrame({"MSE": mse_list, "RMSE": rmse_list, "r2_SCORE": r2_list})

            st.write(res)
            st.balloons()
        else:
            st.error(f"Kolom '{target_column}' tidak ditemukan dalam dataset.")

# Jalankan aplikasi Streamlit
if __name__ == '__main__':
    # Masukkan authtoken Anda di sini
    authtoken = "2iJyVmdEVq1QrJmRETFGsDMgL9W_3W221G4nNav1CVd6ggxBJ"
    ngrok.set_auth_token(authtoken)

    # Buka terowongan ke aplikasi Streamlit
    public_url = ngrok.connect(port='8501')
    print(f"Streamlit app running at: {public_url}")

    # Jalankan aplikasi Streamlit
    process = subprocess.Popen(['streamlit', 'run', 'lulusan.py'])

    # Tampilkan URL aplikasi Streamlit
    print(f"Streamlit app running at: {public_url}")


Overwriting lulusan.py


In [ ]:
!streamlit run lulusan.py &>/dev/null&

In [ ]:
from pyngrok import ngrok

In [ ]:
pip install --upgrade pyngrok

In [32]:
from pyngrok import ngrok

config = {
    "addr": "8501",
    "proto": "http"
}
public_url = ngrok.connect(**config)

In [41]:
from pyngrok import ngrok
ngrok.set_auth_token("2iJyVmdEVq1QrJmRETFGsDMgL9W_3W221G4nNav1CVd6ggxBJ")

In [ ]:
!streamlit run lulusan.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.234.51.73:8501

npx: installed 22 in 2.294s
your url is: https://fancy-rocks-hear.loca.lt
t=2024-06-24T09:59:00+0000 lvl=warn msg="invalid tunnel configuration" pg=/api/tunnels id=54629f6fc73b180e err="yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"
2024-06-24 09:59:00.063 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/ngrok.py", line 518, in api_request
    response = urlopen(request, encoded_data, timeout)
  File "/usr/lib/python3.10/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.10/urllib/request.py", line 525, in open
    response = meth(req, response)
  File "/usr/lib/python3.10/urllib/request.py", line 634, in http_response
    response = self.parent.error(
  F